In [ ]:
df = pd.read_csv('attention-analysis/preprocessed_data_extra.csv')
features = [
    'face_movement', 'body_movement', 'eye_openness_rate',
    'eye_direction_x', 'eye_direction_y', 'mouth_openness_rate',
    'yaw_angle', 'pitch_angle', 'roll_angle'
]
INPUT_SIZE = len(features)
SEQ_LEN = 30

scaler = MinMaxScaler()
df[features] = scaler.fit_transform(df[features])

X_seq, y_seq = [], []
for user_id in df['id'].unique():
    user_df = df[df['id'] == user_id].copy()
    for i in range(len(user_df) - SEQ_LEN):
        seq = user_df[features].iloc[i:i + SEQ_LEN].values
        label = user_df['isAttentive'].iloc[i + SEQ_LEN - 1]
        X_seq.append(seq)
        y_seq.append(int(label))

X_seq, y_seq = np.array(X_seq), np.array(y_seq)
X_flat = X_seq.reshape((X_seq.shape[0], -1))
rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(X_flat, y_seq)
X_seq_balanced = X_resampled.reshape((-1, SEQ_LEN, INPUT_SIZE))
y_seq_balanced = y_resampled

X_train_val, X_test, y_train_val, y_test = train_test_split(
    X_seq_balanced, y_seq_balanced, test_size=0.2, stratify=y_seq_balanced, random_state=42
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.2, stratify=y_train_val, random_state=42
)

class SequenceDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# --- 2. TimesNet Model Tanımı (Değişiklik yok) ---
class TimesBlock(nn.Module):
    def __init__(self, seq_len, num_kernels, d_model, d_ff):
        super(TimesBlock, self).__init__()
        self.seq_len = seq_len
        self.num_kernels = num_kernels
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=d_model, out_channels=d_ff, kernel_size=(1, 3), padding=(0, 1)),
            nn.GELU(),
            nn.Conv2d(in_channels=d_ff, out_channels=d_model, kernel_size=(1, 3), padding=(0, 1))
        )
        self.layer_norm = nn.LayerNorm(d_model)

    def forward(self, x):
        B, T, N = x.size()
        fft_series = torch.fft.rfft(x, dim=1)
        power = torch.abs(fft_series)
        amplitudes = power[:, 1:T//2+1]
        top_amplitudes, top_indices = torch.topk(amplitudes.mean(0).mean(-1), self.num_kernels)
        periods = T // (top_indices + 1)
        res = torch.zeros_like(x)
        for p in periods:
            remainder = self.seq_len % p
            if remainder != 0:
                target_len = self.seq_len + (p - remainder)
                padding = (0, 0, 0, target_len - self.seq_len)
                padded_x = F.pad(x, padding, mode='replicate')
            else:
                target_len = self.seq_len
                padded_x = x
            reshaped_x = padded_x.reshape(B, target_len // p, p, N).permute(0, 3, 2, 1)
            conv_out = self.conv(reshaped_x)
            unreshaped_out = conv_out.permute(0, 3, 2, 1).reshape(B, target_len, N)
            output = unreshaped_out[:, :self.seq_len, :]
            res += output
        res = res + x
        return self.layer_norm(res)

class TimesNetClassifier(nn.Module):
    def __init__(self, seq_len, input_size, num_classes, n_layers, num_kernels, d_model, d_ff, dropout):
        super(TimesNetClassifier, self).__init__()
        self.embedding = nn.Linear(input_size, d_model)
        self.times_blocks = nn.ModuleList([
            TimesBlock(seq_len, num_kernels, d_model, d_ff) for _ in range(n_layers)
        ])
        self.pooling = nn.AdaptiveAvgPool1d(1)
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(d_model, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        for block in self.times_blocks:
            x = block(x)
        x = x.permute(0, 2, 1)
        x = self.pooling(x).squeeze(-1)
        x = self.dropout(x)
        return self.classifier(x)

# --- 3. En İyi Parametrelerin Tanımlanması ---
# Optuna'dan elde edilen en iyi parametreleri buraya doğrudan yazıyoruz.
best_params = {
    'learning_rate': 0.0008975037165212926,
    'd_model': 32,
    'n_layers': 3,
    'num_kernels': 5,
    'dropout': 0.38232996193080226,
    'batch_size': 16
}
print("Kullanılan en iyi parametreler:")
print(best_params)

# --- 4. Final Modelin Eğitimi, Testi ve EZBERLEME ANALİZİ ---
print("\n--- En İyi Parametrelerle Final Model Eğitiliyor ---")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Veri Yükleyicileri
X_train_final = np.concatenate((X_train, X_val), axis=0)
y_train_final = np.concatenate((y_train, y_val), axis=0)
final_train_dataset = SequenceDataset(X_train_final, y_train_final)
val_dataset = SequenceDataset(X_val, y_val) # Validation seti ayrıca kullanılacak
test_dataset = SequenceDataset(X_test, y_test)

final_train_loader = DataLoader(final_train_dataset, batch_size=best_params['batch_size'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=best_params['batch_size'])
test_loader = DataLoader(test_dataset, batch_size=best_params['batch_size'])

# Model, Optimizasyon ve Planlayıcı
final_model = TimesNetClassifier(
    seq_len=SEQ_LEN, input_size=INPUT_SIZE, num_classes=2,
    n_layers=best_params['n_layers'], num_kernels=best_params['num_kernels'],
    d_model=best_params['d_model'], d_ff=best_params['d_model']*4,
    dropout=best_params['dropout']
).to(device)

optimizer = torch.optim.Adam(final_model.parameters(), lr=best_params['learning_rate'])
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.2, patience=10, verbose=True)

# Kayıpları (loss) takip etmek için listeler
train_losses = []
val_losses = []
FINAL_EPOCHS = 150

for epoch in range(FINAL_EPOCHS):
    # Eğitim Aşaması
    final_model.train()
    total_train_loss = 0
    for xb, yb in final_train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        pred = final_model(xb)
        loss = criterion(pred, yb)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()
    avg_train_loss = total_train_loss / len(final_train_loader)
    train_losses.append(avg_train_loss)
    
    # Doğrulama (Validation) Aşaması
    final_model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            pred = final_model(xb)
            loss = criterion(pred, yb)
            total_val_loss += loss.item()
    avg_val_loss = total_val_loss / len(val_loader)
    val_losses.append(avg_val_loss)

    scheduler.step(avg_val_loss) # En iyi pratik: Scheduler'ı validation loss ile beslemek
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1:03d}/{FINAL_EPOCHS} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

# --- 5. Ezberleme (Overfitting) Analizi için Grafik ---
plt.figure(figsize=(12, 6))
plt.plot(train_losses, label='Eğitim Kaybı (Training Loss)')
plt.plot(val_losses, label='Doğrulama Kaybı (Validation Loss)')
plt.title('Eğitim ve Doğrulama Kaybı Grafiği')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

# --- 6. Final Test Raporu ---
print("\n--- Final Test Raporu ---")
final_model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for xb, yb in test_loader:
        xb = xb.to(device)
        outputs = final_model(xb)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        y_true.extend(yb.numpy())
        y_pred.extend(preds)

print(classification_report(y_true, y_pred, target_names=['Dikkatsiz (0)', 'Dikkatli (1)']))
print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred))

In [ ]:

--- Final Test Raporu ---
               precision    recall  f1-score   support

Dikkatsiz (0)       0.91      0.95      0.93       144
 Dikkatli (1)       0.95      0.91      0.93       143

     accuracy                           0.93       287
    macro avg       0.93      0.93      0.93       287
 weighted avg       0.93      0.93      0.93       287


Confusion Matrix:
[[137   7]
 [ 13 130]]